In [2]:
%pip install dadmatools spacy_stanza hazm ipywidgets

  Using cached dadmatools-1.4.0-py3-none-any.whl (848 kB)
  Using cached spacy_stanza-1.0.1-py3-none-any.whl (9.7 kB)
  Using cached hazm-0.7.0-py3-none-any.whl (316 kB)
  Using cached ipywidgets-7.7.0-py2.py3-none-any.whl (123 kB)
  Using cached fasttext-0.9.2.tar.gz (68 kB)
  Using cached Deprecated-1.2.6-py2.py3-none-any.whl (8.1 kB)
  Using cached html2text-2020.1.16-py3-none-any.whl (32 kB)
  Using cached pytorch_transformers-1.2.0-py3-none-any.whl (176 kB)
  Using cached py7zr-0.17.2-py3-none-any.whl (68 kB)
  Using cached transformers-4.9.1-py3-none-any.whl (2.6 MB)
  Using cached tf_estimator_nightly-2.8.0.dev2021122109-py2.py3-none-any.whl (462 kB)
  Using cached spacy-3.0.0-cp39-cp39-manylinux2014_x86_64.whl (12.5 MB)
  Using cached h5py-3.6.0-cp39-cp39-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (4.5 MB)
  Using cached hyperopt-0.2.7-py2.py3-none-any.whl (1.6 MB)
  Using cached folium-0.2.1.tar.gz (69 kB)
  Using cached gensim-4.1.2-cp39-cp39-manylinux_2_12_x86_64.manylin

### Read symbols

In [3]:
import stanza
import spacy_stanza
import re
import json
import pandas as pd
from spacy.tokens import Span
from spacy import displacy
from spacy.symbols import VERB, NOUN, AUX, ADV, ADP


In [4]:
stanza.install_corenlp()
stanza.download("fa")
stanza_nlp = spacy_stanza.load_pipeline("fa")


2022-04-15 21:24:16 INFO: Installing CoreNLP package into /home/amin/stanza_corenlp...


2022-04-15 22:05:37 INFO: Downloading default packages for language: fa (Persian)...


2022-04-15 22:08:37 INFO: Finished downloading models and saved to /home/amin/stanza_resources.
2022-04-15 22:08:37 INFO: Loading these models for language: fa (Persian):
| Processor | Package |
-----------------------
| tokenize  | perdt   |
| mwt       | perdt   |
| pos       | perdt   |
| lemma     | perdt   |
| depparse  | perdt   |

2022-04-15 22:08:37 INFO: Use device: gpu
2022-04-15 22:08:37 INFO: Loading: tokenize
2022-04-15 22:08:44 INFO: Loading: mwt
2022-04-15 22:08:44 INFO: Loading: pos
2022-04-15 22:08:45 INFO: Loading: lemma
2022-04-15 22:08:45 INFO: Loading: depparse
2022-04-15 22:08:45 INFO: Done loading processors!


In [20]:
df = pd.read_csv("symbols_info.csv", index_col=0)
df["Corp"] = df["Corp. Title"].apply(
    lambda word: re.sub(r"\(.*\)|(ح\.)|(ح\s\.)", "", word).strip()
)
df = df[["Symbol", "Corp"]]
mask = df["Corp"] == df["Symbol"]
df["Corp"][mask] = "شرکت " + df["Corp"]
df.head(n=10)


,Symbol,Corp
0,ثملی,بین المللی ساختمان و صنعت ملی
1,اخابرح,مخابرات ایران
2,شتولیح,تولی‌پرس‌
3,دسبحانح,سبحان دارو
4,وزمین,بانک ایران زمین
5,سفارسح,سیمان فارس و خوزستان
6,فنوالح,نورد آلومینیوم‌
7,غالبرح,لبنیات ‌کالبر
8,خساپاح,سایپا
9,مبین,مبین انرژی خلیج فارس


In [21]:
events = [
    "سود",
    "ضرر",
    "مثبت",
    "منفی",
    "صعود",
    "نزول",
    "افزایش",
    "کاهش",
    "رشد",
    "ریزش",
    "افشا",
    "افشا ب",
    "افشا الف",
    "اصلاح",
    "افت",
    "نوسان",
    "نوسانات",
    "سقوط",
    "تقسیم سود",
    "تعیین نرخ",
    "عرضه اولیه",
    "صف خرید",
    "صف فروش",
    "خروج سرمایه",
    "افزایش سرمایه",
    "خروج پول",
    "مجمع عمومی",
]
announcements = [
    "اطلاعیه",
    "گزارش",
    "اعلامیه",
    "اعلانیه",
]
analyse = [
    "سهم رانتی",
    "تحلیل تکنیکال",
    "تحلیل فاندامنتال",
    "کندل استیک",
    "سیگنال",
    "تکنیکال",
    "کندل",
    "تیک",
    "سرشانه",
    "مقاومت",
    "حمایت",
    "کراس",
    "واگرایی",
    "اندیکاتور",
    "سقف",
    "کف",
    "قیبوناچی",
    "فیبوناتچی",
    "فیبو",
    "کانال",
    "کف‌‌سازی",
    "کف سازی",
    "رنج",
]
characters = [
    "بازیگر",
    "حقیقی",
    "حقوقی",
    "بازی‌گردان",
    "نوسان‌گیر",
]


In [22]:
from dadmatools.models.normalizer import Normalizer as N1
from hazm import Normalizer as N2

normalizer1 = N1(
    full_cleaning=False,
    unify_chars=True,
    refine_punc_spacing=True,
    remove_extra_space=True,
    remove_puncs=False,
    remove_html=False,
    remove_stop_word=False,
    replace_email_with="<EMAIL>",
    replace_number_with=None,
    replace_url_with="<URL",
    replace_mobile_number_with="<MOBILE_NUMBER>",
    replace_emoji_with="<EMOJI>",
    replace_home_number_with="<HOME_NUMBER>",
)
normalizer2 = N2()


In [23]:
from spacy.matcher import Matcher

matcher: Matcher = Matcher(stanza_nlp.vocab, validate=True)


def add_patterns(matcher, keywords, class_name):
    keyword_patterns = []
    r = "(ی|ای|یی|ها|هایی|های)?"
    for keyword in keywords:
        keyword_tokens = stanza_nlp.tokenizer(keyword)

        pre_det = ["این", "هر", "چند", "چندین", "آن", "همین", "همان", "چنین", "چنین"]
        if len(keyword_tokens) == 1:
            token1_keyword = keyword_tokens[0].text
            pattern = [
                {"TEXT": {"IN": pre_det}, "OP": "?"},
                {"TEXT": {"REGEX": rf"\b{token1_keyword}{r}\b"}},
                {"TEXT": {"IN": ["ای", "ی", "ها", "های", "هایی"]}, "OP": "?"},
            ]

        elif len(keyword_tokens) == 2:
            token1_keyword = keyword_tokens[0].text
            token2_keyword = keyword_tokens[1].text
            pattern = [
                {"TEXT": {"IN": pre_det}, "OP": "?"},
                {"TEXT": {"REGEX": rf"\b{token1_keyword}{r}\b"}},
                {"TEXT": {"IN": ["ای", "ی", "ها", "های", "هایی"]}, "OP": "?"},
                {"TEXT": {"IN": pre_det}, "OP": "?"},
                {"TEXT": {"REGEX": rf"\b{token2_keyword}{r}\b"}},
                {"TEXT": {"IN": ["ای", "ی", "ها", "های", "هایی"]}, "OP": "?"},
            ]

        keyword_patterns.append(pattern)

    matcher.add(class_name, keyword_patterns, greedy="LONGEST")


SYMBOL = "SYMBOL"
CORP = "CORP"
EVENT = "EVENT"
ANALYSE = "ANALYSE"
ANNOUNCE = "ANNOUNCE"
CHARACTERS = "CHARACTERS"

add_patterns(matcher, events, "EVENT")
add_patterns(matcher, analyse, "ANALYSE")
add_patterns(matcher, announcements, "ANNOUNCE")
add_patterns(matcher, characters, "CHARACTERS")


In [24]:
symbols = df["Symbol"].tolist()
symbols = list(map(lambda x: x.replace(".", "\\."), symbols))

corporations = df["Corp"].tolist()
corporations = list(map(lambda x: x.replace(".", "\\."), corporations))


In [25]:
def convert_subtree_to_str(token, remove_ADP=False, is_parent=True):
    extracted_event = ""
    start = None
    end = None
    for x in token.subtree:
        if x.dep_ == "case" and start is None and is_parent:
            continue
        if x.pos == ADV:
            continue
        if remove_ADP and x.pos == ADP and is_parent:
            if x.head.text == token.text:
                if not start is None:
                    print(
                        "WARNING ------------------> check the convert subtree to str"
                    )
                continue

        extracted_event += str(x) + " "
        if start is None:
            start = x.idx
        end = x.idx + len(x.text)

    extracted_event = extracted_event[:-1]
    return extracted_event, start, end


TRANSLATE = {
    SYMBOL: "نماد",
    CORP: "شرکت",
    EVENT: "واقعه",
    ANNOUNCE: "اعلان",
    ANALYSE: "تحلیل",
    CHARACTERS: "شخصیت",
}


def create_output(OUTPUT, output_type, marker, span, **kwargs):
    # print(f'span :{span} , text: "{text[span[0]:span[1]]}" ')
    persian_output_type = TRANSLATE[output_type]
    defaults = {
        "type": persian_output_type,
        "marker": marker,
        "span": span,
    }
    OUTPUT[output_type].append({**defaults, **kwargs})

    return {**defaults, **kwargs}


def concat_tokens(tokens):
    concat = ""
    start = None
    end = None
    for token in tokens:
        if start is None:
            start = token.idx
        substring, s, e = convert_subtree_to_str(token, is_parent=False)
        end = e
        concat += substring + " "
    concat = concat[:-1]
    return concat, start, end


def get_root_string(main_noun):
    left_childs = list(
        filter(lambda x: not x.dep_ in ["cop", "nsubj"], main_noun.lefts)
    )
    right_childs = list(
        filter(lambda x: not x.dep_ in ["cop", "nsubj"], main_noun.rights)
    )
    extracted_event = ""
    start = None
    end = None
    if left_childs:
        substring, s, e = concat_tokens(left_childs)
        if start is None:
            start = s
        end = e
        extracted_event += substring + " "
    extracted_event += main_noun.text
    if start is None:
        start = main_noun.idx
    end = main_noun.idx + len(main_noun.text)

    if right_childs:
        substring, s, e = concat_tokens(right_childs)
        extracted_event += " " + substring
        end = e
    return extracted_event, start, end


In [26]:
def set_ents(symbols, corporations, doc, matcher, text):
    symbols_expression = "|".join(symbols)
    corporation_expression = "|".join(corporations)

    symbol_spans = list(
        map(
            lambda match: doc.char_span(*match.span(), label=SYMBOL),
            re.finditer(symbols_expression, text),
        )
    )

    symbol_spans = list(filter(lambda span: span is not None, symbol_spans))

    corporation_spans = list(
        map(
            lambda match: doc.char_span(*match.span(), label=CORP),
            re.finditer(corporation_expression, text),
        )
    )

    corporation_spans = list(filter(lambda span: span is not None, corporation_spans))

    term_spans = list(
        map(
            lambda match: Span(
                doc, match[1], match[2], label=stanza_nlp.vocab.strings[match[0]]
            ),
            matcher(doc),
        )
    )

    term_spans = list(filter(lambda span: span is not None, term_spans))

    spans = symbol_spans + corporation_spans + term_spans
    doc.set_ents(spans)

    with doc.retokenize() as retokenizer:
        attrs = {"POS": "NOUN"}
        for span in symbol_spans:
            retokenizer.merge(span, attrs)

    with doc.retokenize() as retokenizer:
        attrs = {"POS": "NOUN"}
        for span in corporation_spans:
            retokenizer.merge(span, attrs)

    with doc.retokenize() as retokenizer:
        attrs = {"POS": "NOUN"}
        for span in term_spans:
            retokenizer.merge(span, attrs)

    return doc


In [27]:
def find_symbols_and_corporations(doc, output):
    symbol_entities = list(filter(lambda ent: ent.label_ == SYMBOL, doc.ents))

    for symbol_ent in symbol_entities:
        token = symbol_ent[0]
        start = token.idx
        end = token.idx + len(token.text)
        create_output(output, SYMBOL, token.text, (start, end))

    corporation_entities = list(filter(lambda ent: ent.label_ == CORP, doc.ents))

    for corporation_ent in corporation_entities:
        token = corporation_ent[0]
        start = token.idx
        end = token.idx + len(token.text)
        create_output(output, CORP, token.text, (start, end))


In [28]:
def find_events(doc, output):
    term_entities = list(filter(lambda ent: ent.label_ not in [SYMBOL, CORP], doc.ents))

    for term_ent in term_entities:
        token = term_ent[0]
        # print('------------')
        # print('token is: ', token.text)

        if token.dep_ == "compound:lvc":
            # print('parent is: ', token.head)
            if token.head.pos == VERB:
                # print('parent was a verb')
                # print('children of parent: ', list(token.head.children))
                subject = None
                for child in token.head.children:
                    if child.dep_ == "nsubj":
                        # print('Found a subject, outputing subtree of subject as subject')
                        subject, start_subj, end_subj = convert_subtree_to_str(child)
                        break
                    else:
                        print("NOT IMP - child dep is: ", child.dep_)
                        pass
                else:
                    print("didn't found any subj")
                    pass

                token_string, start, end = convert_subtree_to_str(token)
                end = token.head.idx + len(token.head.text)
                extracted_event = token_string + " " + token.head.text
                if subject:
                    create_output(
                        output,
                        token.ent_type_,
                        extracted_event,
                        (start, end),
                        subject=subject,
                        span_subject=(start_subj, end_subj),
                    )
                else:
                    create_output(
                        output, token.ent_type_, extracted_event, (start, end)
                    )
            else:
                extracted_event, start, end = convert_subtree_to_str(token)
                create_output(output, token.ent_type_, extracted_event, (start, end))

        elif token.dep_ == "nmod" or token.dep_ == "amod":
            # print('type is: ', token.dep_)
            # print('parent is: ', token.head)

            # print('finding parent that is not nmod or amod')
            main_noun = token
            ## print('1', main_noun, main_noun.dep_, main_noun.head.pos_)
            while (
                main_noun.dep_ == "nmod" or main_noun.dep_ == "amod"
            ) and main_noun.head.pos == NOUN:
                main_noun = main_noun.head
                ## print('2', main_noun, main_noun.dep_, main_noun.head.pos_)
            if main_noun.pos != NOUN:
                print("WARNING ------------------> check code in else: dep = nmod amod")
                continue

            # print('Found a Noun parent that is not nmod or amod')

            if main_noun.dep_ == "root":
                # print('Noun parent is root, text: ', main_noun.text)
                children = list(main_noun.children)
                # print('children are: ', children)
                # print('left chilren are: ', list(main_noun.lefts))
                # print('right chilren are: ', list(main_noun.rights))
                for child in children:
                    # print('child text: ', child.text, ' pos: ', child.pos_, ' dep_: ', child.dep_)
                    if child.dep_ == "cop" and child.pos == AUX:
                        extracted_event, start, end = get_root_string(main_noun)
                        create_output(
                            output, token.ent_type_, extracted_event, (start, end)
                        )
                        break
                else:
                    # TODO: this is of no use
                    extracted_event, start, end = get_root_string(main_noun)
                    create_output(
                        output, token.ent_type_, extracted_event, (start, end)
                    )

            else:
                extracted_event, start, end = convert_subtree_to_str(
                    main_noun, remove_ADP=True
                )
                # extracted_event, start, end = convert_subtree_to_str(main_noun, not_in=['ADP'])

                create_output(output, token.ent_type_, extracted_event, (start, end))

        elif token.dep_ == "root":
            extracted_event, start, end = get_root_string(token)
            create_output(output, token.ent_type_, extracted_event, (start, end))

        else:
            # print('type is: other')
            extracted_event, start, end = convert_subtree_to_str(token, remove_ADP=True)

            # print(text[start:end])
            create_output(output, token.ent_type_, extracted_event, (start, end))


In [29]:
def has_intersection(first, second):
    if first[0] < second[0]:
        if first[1] < second[0]:
            return False
        else:
            return True
    else:
        if first[0] > second[1]:
            return False
        else:
            return True


def combine(first, second, text):
    start = min(first["span"][0], second["span"][0])
    end = max(first["span"][1], second["span"][1])
    new_span = (start, end)
    new_marker = text[start:end]
    return new_span, new_marker


def remove_type_overlap(match_list, text):
    n = len(match_list)
    if n == 0:
        return []
    i = 0
    j = 1
    while i < n and j < n:
        first = match_list[i]
        second = match_list[j]
        if has_intersection(first["span"], second["span"]):
            new_span, new_marker = combine(first, second, text)
            match_list[i]["span"] = new_span
            match_list[i]["marker"] = new_marker
            match_list[j] = None
        else:
            i = j
        j += 1

    match_list = list(filter(lambda x: not x is None, match_list))
    return match_list


def remove_span_overlap(OUTPUT, text):
    for key in OUTPUT.keys():
        OUTPUT[key] = remove_type_overlap(OUTPUT[key], text)
    return OUTPUT


def print_output(OUTPUT, text):
    for key in OUTPUT.keys():
        for o in OUTPUT[key]:
            # print('----')
            # print(text[o['span'][0]:o['span'][1]])
            print(json.dumps(o, ensure_ascii=False, indent=2))


In [30]:
def run(*texts):
    for text_index, text in enumerate(texts):
        output = {
            SYMBOL: [],
            CORP: [],
            EVENT: [],
            ANNOUNCE: [],
            ANALYSE: [],
            CHARACTERS: [],
        }

        text = normalizer1.normalize(text)
        text = normalizer2.normalize(text)

        print(
            f"---------------------------- input {text_index}----------------------------------------------"
        )
        print(f"Normalized input: {text}")

        doc = stanza_nlp(text)

        doc = set_ents(symbols, corporations, doc, matcher, text)

        find_symbols_and_corporations(doc, output)

        find_events(doc, output)

        # print('Entities: ...')
        # displacy.render(doc, style='ent', jupyter=True)

        # print('Dependency Tree: ...')
        # displacy.render(doc, style='dep', jupyter=True)

        output = remove_span_overlap(output, text)
        print_output(output, text)


In [31]:
examples = [
    "برکت امروز اطلاعیه‌ای مهم منتشر میکند.",
    "نماد برکت امروز عرضه‌ی اولیه خیلی خوبی داره.",
    "نماد برکت امروز عرضه ی اولیه خیلی خوبی داره.",
    "نماد برکت امروز عرضه ی اولیه دارد.",
    "عرضه های اولیه امروز خوب هستند.",
    "عرضه اولیه‌های امروز خوب هستند.",
    "این عرضه‌ی اولیه خیلی خوبه",
    "رشد قیمت‌ها باعث ایجاد صف خرید در سهم پرشیا شد",
    "یک نکته‌ی تکنیکالی هم در صورت دستکاری نشدن اضافه کنم، کندلی که روز سه شنبه‌ی گذشته ثبت کرد کامل است",
    "آ س پ امروز روند مثبتی داشت.",
    "آمریکا موجب ریزش بازار شد",
    "ارزش سهام مخابرات ایران امروز کاهش زیادی یافت.‎",
    "قیمت سهام زیاد است. رشد قیمت‌ها باعث کاهش قیمت شد. ریزش بازار هم به همین دلیل بود.",
]


run(*examples)


---------------------------- input 0----------------------------------------------
Normalized input: برکت امروز اطلاعیه‌ای مهم منتشر میکند.
{
  "type": "نماد",
  "marker": "برکت",
  "span": [
    0,
    4
  ]
}
{
  "type": "اعلان",
  "marker": "اطلاعیه‌ای مهم",
  "span": [
    11,
    25
  ]
}
---------------------------- input 1----------------------------------------------
Normalized input: نماد برکت امروز عرضه‌ی اولیه خیلی خوبی داره.
{
  "type": "نماد",
  "marker": "برکت",
  "span": [
    5,
    9
  ]
}
{
  "type": "واقعه",
  "marker": "عرضه‌ی اولیه خوبی",
  "span": [
    16,
    38
  ]
}
---------------------------- input 2----------------------------------------------
Normalized input: نماد برکت امروز عرضه‌ی اولیه خیلی خوبی داره.
{
  "type": "نماد",
  "marker": "برکت",
  "span": [
    5,
    9
  ]
}
{
  "type": "واقعه",
  "marker": "عرضه‌ی اولیه خوبی",
  "span": [
    16,
    38
  ]
}
---------------------------- input 3----------------------------------------------
Normalized inp